# Exponential speedup with the Deutsch-Jozsa algorithm

The Deutsch-Jozsa algorithm[[1](#DJWiki)], named after David Deutsch and Richard Jozsa, is one of the fundamental and first quantum algorithms showing exponential speedup over their classical counterpart$^*$. While it has no practical applicative usage, it serves as a toy model for quantum computing, demonstrating how the concepts of super-position and interference enable quantum algorithms to overperform classical ones.

$^*$ The exponential speedup is in the oracle complexity setting. In addition, it only refers to deterministic classical machines (see comment below).

The algorithm treats the following problem:
* Consider a black-box boolean function $f(x)$ that acts on the integers in the range $[0, 2^{n}-1]$.
* It is guaranteed that the function is either constant or balanced (for half of the values it is 1 and for the other half it is 0).
* The goal is to find, in a deterministic way, whether the function is constant or balanced.

A brief mathematical explanation of the algorithm appears in Section 5.

## 1. Implementing the Deutsch-Jozsa algorithm

Start with defining a `deutsch_jozsa` quantum function, whose arguments are a quantum function for the black-box $f(x)$, and a quantum variable on which it acts, $x$. The Deutsch-Jozsa algorithm is composed of three quantum blocks (see Figure 1): a Hadamard transform, an arithmetic oracle for the black-box function, and another Hadamard transform. 

In [2]:
from classiq import (
    H,
    Output,
    QBit,
    QCallable,
    QFunc,
    QNum,
    X,
    allocate,
    hadamard_transform,
    within_apply,
)


@QFunc
def prep_minus(out: Output[QBit]) -> None:
    allocate(1, out)
    X(out)
    H(out)


@QFunc
def my_oracle(predicate: QCallable[QNum, QBit], target: QNum) -> None:
    aux = QBit("aux")
    within_apply(compute=lambda: prep_minus(aux), action=lambda: predicate(target, aux))


@QFunc
def deutsch_jozsa(predicate: QCallable[QNum, QBit], x: QNum) -> None:
    hadamard_transform(x)
    my_oracle(predicate=lambda x, y: predicate(x, y), target=x)
    hadamard_transform(x)

The classical part of the algorithm reads: The probability of measuring the $|0\rangle_n$ state is 1 if the function is constant and 0 if it is balanced. 
Define a classical function that gets the execution results from running quantum part and returns whether the function is constant or balanced:

In [3]:
def post_process_deutsch_jozsa(parsed_results):
    if len(parsed_results) == 1:
        if 0 not in parsed_results:
            print("The function is balanced")
        else:
            print("The function is constant")
    else:
        print(
            "cannot decide as more than one output was measured, the distribution is:",
            parsed_results,
        )

## 2. Applying the algorithm on a simple oracle

Start with a simple example on $n=4$ qubits, and $f(x)= x >7$. Classicaly, in the worst case, the function should be evaluated $2^{n-1}+1=9$ times. However, with the Deutsch-Jozsa algorithm, this function is evaluated only once.

Build a predicate for the specific use case:

In [4]:
@QFunc
def simple_predicate(x: QNum, res: QBit) -> None:
    res ^= x > 7

Define a model by inserting the predicate into the `deutsch_jozsa` function:

In [5]:
from classiq import create_model, show, synthesize

NUM_QUBITS = 4


@QFunc
def main(x: Output[QNum[NUM_QUBITS]]):
    allocate(NUM_QUBITS, x)
    deutsch_jozsa(lambda x, y: simple_predicate(x, y), x)


qmod = create_model(main)
qprog = synthesize(qmod)

Execute and call the classical post process:

In [6]:
from classiq import execute

results = execute(qprog).result()
results_list = [sample.state["x"] for sample in results[0].value.parsed_counts]
post_process_deutsch_jozsa(results_list)

The function is balanced


In [12]:
with open("simple_deutsch_jozsa.qmod", "w") as f:
    f.write(qmod)
show(qprog)

<center>
<img src="https://classiq-docs-images.s3.amazonaws.com/deutsch_josza_closed.png" style="width:100%">
<figcaption align = "middle"> Figure 1. The Deutsch-Jozsa algorithm </figcaption>
</center>

## 3. Applying the algorithm on a more complex oracle

Generalizing to more complex scenarios makes no difference for modeling. Take a complicated function, working with $n=3$: a function $f(x)$ that first takes the maximum between the input Bitwise-Xor with 4 and the input Bitwise-And with 3, then checks whether the result is greater or equal to 4. Can you tell whether the function is balanced or constant?

*This time provide a width bound to the Synthesis engine.*

Follow the three steps as before:

In [8]:
from classiq import Input, QArray, set_constraints
from classiq.model import Constraints
from classiq.qmod.symbolic import max

NUM_QUBITS = 3


@QFunc
def complex_predicate(x: QNum, res: QBit) -> None:
    res ^= max(x ^ 4, x & 3) >= 4


@QFunc
def main(x: Output[QNum[NUM_QUBITS]]):
    allocate(NUM_QUBITS, x)
    deutsch_jozsa(lambda x, y: complex_predicate(x, y), x)


qmod = create_model(main)
qmod = set_constraints(qmod, constraints=Constraints(max_width=19))
qprog = synthesize(qmod)

results = execute(qprog).result()
results_list = [sample.state["x"] for sample in results[0].value.parsed_counts]
post_process_deutsch_jozsa(results_list)

The function is balanced


<center>
<img src="https://classiq-docs-images.s3.amazonaws.com/deutsch_jozsa_opened.png" style="width:100%">
<figcaption align = "middle"> Figure 2. The Deutsch-Jozsa algorithm for the complex example, focusing on oracle implementation </figcaption>
</center>

Visualize the circuit obtained from the synthesis engine. Figure 2 presents the complex structure of the oracle, generated automatically by the Synthesis engine.

In [11]:
show(qprog)

In [10]:
with open("complex_deutsch_jozsa.qmod", "w") as f:
    f.write(qmod)

## 4. Mathematical explanation

A brief summary of the linear algebra behind the Deutsch-Jozsa algorithm. The first Hadamard transformation generates an equal super-position over all the standard basis elements:
$$
|0\rangle_n \xrightarrow[H^{\otimes n}]{} \frac{1}{2^{n/2}}\sum^{2^n-1}_{j=0}|j\rangle_n.
$$
Arithmetic oracle gets a boolean function and adds an $e^{\pi i}=-1$ phase to all states for which the function returns True:
$$
\frac{1}{2^{n/2}}\sum^{2^n-1}_{j=0}|j\rangle_n \xrightarrow[\text{Oracle}(f(j))]{}\frac{1}{2^{n/2}}\sum^{2^n-1}_{j=0}(-1)^{f(j)}|j\rangle_n.
$$
Finally, application of the Hadamard transform, which can be written as $H^{\otimes n}\equiv \frac{1}{2^{n/2}}\sum^{2^n-1}_{k,l=0}(-1)^{k\cdot l} |k\rangle \langle l| $, gives
$$
\frac{1}{2^{n/2}}\sum^{2^n-1}_{j=0}(-1)^{f(j)}|j\rangle  \xrightarrow[H^{\otimes n}]{} 
\sum^{2^n-1}_{k=0} \left(\frac{1}{2^{n}}\sum^{2^n-1}_{j=0}(-1)^{f(j)+j\cdot k} \right) |k\rangle.
$$

The probability of getting the state $|k\rangle = |0\rangle$ is
$$
P(0)=\left|\frac{1}{2^{n}}\sum^{2^n-1}_{j=0}(-1)^{f(j)} \right|^2 =
\left\{
\begin{array}{l l}
1 & \text{if } f(x) \text{ is constant} \\
0 & \text{if } f(x) \text{ is balanced}
\end{array}
\right.
$$

## 5. Comments

Without requiring deterministic determination, namely, allowing application of classical probabilistic algorithm to get the result up to some error, then the exponential speedup is lost: taking $k$ classical evaluations of the function $f$ determines whether the function is constant or balanced, with a probability $1-1/2^k$.

## References

<a id='DJWiki'>[1]</a>: [Deutsch Jozsa (Wikipedia)](https://en.wikipedia.org/wiki/Deutsch%E2%80%93Jozsa_algorithm)
